In [77]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
#set path for my_tools script
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import file_dic, plot_dic
import matplotlib.ticker as plticker
import string


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [93]:
#set file paths
path_annual = '/media/onno/Algemeen/Thesis/GFS_mean/'
path_season = '/media/onno/Algemeen/Thesis/GFS_mean_season/'
path_to_write = '/media/onno/Algemeen/Thesis/Main_Figures/'

#set variables
variablez = ['envelope','phasespeed','u_wind']
#set ax titles
titlez = ['Annual','DJF','JJA']
letterz = np.array(list(string.ascii_lowercase)[:9]).reshape(3,3)
#deifne each lead hour
leads = np.linspace(0,240,11)
#set latitude data for NH and SH to plot
# loc = plticker.MultipleLocator(base=0.25,nbins='auto') 
# this locator puts ticks at regular intervals
latz_hem = [(75,25),(-25,-75)]
for lat_hem in latz_hem:
    #intialize figure
    fig,axz = plt.subplots(3,3,sharey='row',sharex=True,figsize=(10,7.5))
    for k,variable in enumerate(variablez):
        #set relevant axis
        if lat_hem==(75,25):
            latz = [(75,65),(65,55),(55,45),(45,35),(35,25)]
            legendz = ['65-75 °N','55-65 °N','45-55 °N','35-45 °N','25-35 °N']
            colorz = ['#0066ff','#009900','#e6e600','#ff6600','#ff0000']
            hemisphere = 'Northern Hemisphere'
        else:
            latz = [(-25,-35),(-35,-45),(-45,-55),(-55,-65),(-65,-75)]
            legendz = ['25-35 °S','35-45 °S','45-55 °S','55-65 °S','65-75 °S']
            colorz = ['#0066ff','#009900','#e6e600','#ff6600','#ff0000'][::-1]
            hemisphere = 'Southern Hemisphere'
        #create empty lists to add each dataset for each lead day for annual results, DJF and JJA
        datasets_annual = []
        datasets_DJF = []
        datasets_JJA = []
        #loop over lead days
        for i in range(11):
            file_annual = 'GFS_mean_error_{}_day_{:02d}_globe_mars.nc'.format(variable,i)
            file_DJF = 'GFS_mean_error_{}_DJF_day_{:02d}_globe.nc'.format(variable,i)
            file_JJA = 'GFS_mean_error_{}_JJA_day_{:02d}_globe.nc'.format(variable,i)
            #open dataset for each lead day and append to empty lists. 
            #Also only select data between 75 and 25 degrees north and south
            ds_single = xr.open_dataset(path_annual+file_annual,decode_times=False).sel(lat=slice(lat_hem[0],lat_hem[1]))
            datasets_annual.append(ds_single)
            ds_single_DJF = xr.open_dataset(path_season+file_DJF,decode_times=False).sel(lat=slice(lat_hem[0],lat_hem[1]))
            datasets_DJF.append(ds_single_DJF)
            ds_single_JJA = xr.open_dataset(path_season+file_JJA,decode_times=False).sel(lat=slice(lat_hem[0],lat_hem[1]))
            datasets_JJA.append(ds_single_JJA)
        #create 3D dataset with lead time as third dimension
        ds = xr.concat(datasets_annual,dim='lead')
        ds_DJF = xr.concat(datasets_DJF,dim='lead')
        ds_JJA = xr.concat(datasets_JJA,dim='lead')
        
        mean_lat_lead_annual = []
        mean_lat_lead_DJF = []
        mean_lat_lead_JJA = []
        for lat in latz:
        #create empty list for each bin
            mean_lat_each_lead_annual=[]
            mean_lat_each_lead_DJF=[]
            mean_lat_each_lead_JJA=[]
            #loop over each lead time (11 in total: 0 to 240 hours with 24 h intervals)
            for lead in leads:   
                #Calculating the mean compensating for cosine latitude weighting
                ds_lat = ds.sel(lat=slice(lat[0],lat[1]),lead=lead)
                ds_DJF_lat = ds_DJF.sel(lat=slice(lat[0],lat[1]),lead=lead)
                ds_JJA_lat = ds_JJA.sel(lat=slice(lat[0],lat[1]),lead=lead)
                #calculate latitude weighted mean average
                ds_lat_mean = my_tools.weighted_average_area_2D(ds_lat)
                ds_DJF_lat_mean = my_tools.weighted_average_area_2D(ds_DJF_lat)
                ds_JJA_lat_mean = my_tools.weighted_average_area_2D(ds_JJA_lat)
                #append means for all lead days to latitude bin
                mean_lat_each_lead_annual.append(float(ds_lat_mean))
                mean_lat_each_lead_DJF.append(float(ds_DJF_lat_mean))
                mean_lat_each_lead_JJA.append(float(ds_JJA_lat_mean))
            #append each latitude bin to get 2D list    
            mean_lat_lead_annual.append(mean_lat_each_lead_annual)
            mean_lat_lead_DJF.append(mean_lat_each_lead_DJF)
            mean_lat_lead_JJA.append(mean_lat_each_lead_JJA)
        #convert list to numpy array
        mean_lat_lead_annual = np.array(mean_lat_lead_annual)
        mean_lat_lead_DJF = np.array(mean_lat_lead_DJF)
        mean_lat_lead_JJA = np.array(mean_lat_lead_JJA)
        for j in range(3):
            ax = axz[k,j]
            ax.text(x=0,y=1.02,s=letterz[k,j]+')',transform=ax.transAxes,fontweight='bold',fontsize=12,
               verticalalignment='bottom',horizontalalignment='left')
            ax.margins(x=0)
            ax.set_xlim([0,10])
            ax.set_xticks(np.arange(11))
            ax.grid(b=True,which='major',axis='y')
            if k==0:
                ax.set_title(titlez[j])
            if j==0:
                ax.set_ylabel(plot_dic[variable]['label'])
            if k==2:
                ax.set_xlabel('Lead Days')
            for i in range(len(mean_lat_lead_annual)):
                if variable == 'phasespeed':
                    mean_lat_lead_annual[:,0]=np.nan
                    mean_lat_lead_DJF[:,0]=np.nan
                    mean_lat_lead_JJA[:,0]=np.nan
                if j ==0:
                    ax.plot(range(11),mean_lat_lead_annual[i],label=legendz[i],color=colorz[i])
                elif j==1:
                    ax.plot(range(11),mean_lat_lead_DJF[i],label=legendz[i],color=colorz[i])
                else:
                    ax.plot(range(11),mean_lat_lead_JJA[i],label=legendz[i],color=colorz[i])
            if (k==0)&(j==2):
                ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    fig.subplots_adjust(left=0.08,right=0.85,wspace=0.075,hspace=0.2,bottom=0.075)
    if lat_hem == (75,25):
        fig.suptitle('GEFS Forecast Errors Northern Hemisphere')
        fig.savefig(path_to_write+'GEFS_Error_Evolution_NH',dpi=300)
    else:
        fig.suptitle('GEFS Forecast Errors Southern Hemisphere')
        fig.savefig(path_to_write+'GEFS_Error_Evolution_SH',dpi=300)
    


In [94]:
plt.close('all')